<a href="https://colab.research.google.com/github/Jin-jin-jara/Small-Project1--SVHN_3_digits_classification/blob/master/SVHN(basic_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVHN 3 digits classfication

데이터를 전처리 후 기본 CNN모델에 훈련시켰습니다.

In [0]:
%tensorflow_version 2.x
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os

from scipy.io import loadmat
from skimage import color
from skimage import io

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, Dropout, BatchNormalization
from keras import optimizers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [55]:
def load_data(path):
  data = loadmat(path)
  return data['X'], data['y']

x, y = load_data("/content/drive/My Drive/test_32x32.mat")

x.shape, y.shape

((32, 32, 3, 26032), (26032, 1))

In [56]:
x, y = x.transpose((3,0,1,2)), y[:, 0]

x.shape, y.shape

((26032, 32, 32, 3), (26032,))

In [0]:
m = len(y)//2

x_train = x[:m]
y_train = y[:m]
x_test = x[m:m*2]
y_test = y[m:m*2]

In [0]:
def plot_images(img, labels, nrows, ncols):

  fig, axes = plt.subplots(nrows, ncols)
  for i, ax in enumerate(axes.flat):
    if img[i].shape == (32,32,3):
      ax.imshow(img[i])
    else:
      ax.imshow(img[i,:,:,0])
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title(labels[i])

In [61]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

print(np.unique(y_train))
print(np.unique(y_test))

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [0]:
def attach_3_image(x, y):
  m = len(y)
  m = m-m%3
  pad = 6
  x, y = x[:m], y[:m]
  x = x[:,:, pad:-pad] 
  x_attach = np.concatenate((x[::3], x[1::3], x[2::3]), axis=2)
  y_attach = np.reshape(y, [-1,3])
  return x_attach, y_attach

In [64]:
x3_train, y3_train = attach_3_image(x_train, y_train)
x3_test, y3_test = attach_3_image(x_test, y_test)
x3_train.shape, x3_test.shape, y3_train.shape, y3_test.shape

((4338, 32, 60, 3), (4338, 32, 60, 3), (4338, 3), (4338, 3))

In [66]:
def rgb2gray(images):
  return np.expand_dims(np.dot(images, [0.2126, 0.7152, 0.0722]), axis=3)

train_grayscale = rgb2gray(x3_train).astype(np.float32)
x3_test = rgb2gray(x3_test).astype(np.float32)
train_grayscale.shape, x3_test.shape

((4338, 32, 60, 1), (4338, 32, 60, 1))

In [0]:
train_mean = np.mean(train_grayscale, axis=0)
train_std = np.std(train_grayscale, axis=0)
train_grayscale_norm = (train_grayscale - train_mean) / train_std

test_mean = np.mean(x3_test, axis=0)
test_std = np.std(x3_test, axis=0)
test_grayscale_norm = (x3_test - test_mean) / test_std

In [68]:
train_grayscale_norm.shape, y3_train.shape, test_grayscale_norm.shape, y3_test.shape

((4338, 32, 60, 1), (4338, 3), (4338, 32, 60, 1), (4338, 3))

## CNN model

In [0]:
BATCH_SIZE = 128

train_ds = tf.data.Dataset.from_tensor_slices((train_grayscale_norm, y3_train)).shuffle(10000).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((test_grayscale_norm, y3_test)).shuffle(10000).batch(BATCH_SIZE)

In [0]:
class MyModel(Model): 
  def __init__(self):
    super(MyModel, self).__init__()
    self.k = 10     
    self.seq = 3    
    self.conv1 = Conv2D(32, 3, padding="same", activation="relu")
    self.pool1 = MaxPool2D(2, padding="same")
    self.conv2 = Conv2D(32, 3, padding='same', activation="relu")  
    self.pool2 = MaxPool2D(2, padding="same")                       
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(self.k*self.seq)  

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)  
    x = self.conv2(x)
    x = self.pool2(x)     
    x = self.flatten(x) 
    x = self.d1(x)   
    h = self.d2(x) 
    h = tf.reshape(h, [-1, self.seq, self.k]) 
    h = tf.nn.softmax(h, axis=2)   
    return h

model = MyModel()

In [0]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name="train_acc")

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_acc")

In [0]:
@tf.function
def train_step(images, labels):
  y_hot = tf.one_hot(labels, depth=10, axis=-1)
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(y_hot, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_hot, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  y_hot = tf.one_hot(labels, depth=10, axis=-1)
  predictions = model(images)
  t_loss = loss_object(y_hot, predictions)

  test_loss(t_loss)
  test_accuracy(y_hot, predictions)

In [89]:
EPOCHS = 500

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)
  
  for test_images, labels in test_ds:
    test_step(test_images, labels)
  
  template = "에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}"
  
  if epoch%40==0:
    print(template.format(epoch,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

에포크: 0, 손실: 2.23038649559021, 정확도: 18.84893226623535, 테스트 손실: 2.1925623416900635, 테스트 정확도: 20.623943328857422
에포크: 40, 손실: 0.2970573306083679, 정확도: 90.68038940429688, 테스트 손실: 1.5470421314239502, 테스트 정확도: 68.62478637695312
에포크: 80, 손실: 0.15067939460277557, 정확도: 95.28266906738281, 테스트 손실: 1.8767503499984741, 테스트 정확도: 70.23480987548828
에포크: 120, 손실: 0.1009187325835228, 정확도: 96.84211730957031, 테스트 손실: 2.0939059257507324, 테스트 정확도: 70.7218017578125
에포크: 160, 손실: 0.07585880905389786, 정확도: 97.6266860961914, 테스트 손실: 2.2636795043945312, 테스트 정확도: 70.94261932373047
에포크: 200, 손실: 0.060767777264118195, 정확도: 98.09898376464844, 테스트 손실: 2.4084596633911133, 테스트 정확도: 71.069580078125
에포크: 240, 손실: 0.05068189650774002, 정확도: 98.41450500488281, 테스트 손실: 2.538325071334839, 테스트 정확도: 71.14674377441406
에포크: 280, 손실: 0.04346739128232002, 정확도: 98.64019775390625, 테스트 손실: 2.6586077213287354, 테스트 정확도: 71.19824981689453
에포크: 320, 손실: 0.03805089369416237, 정확도: 98.80964660644531, 테스트 손실: 2.772386074066162, 테스트 정확도: 71.23